In [49]:
%cd ../

/content/drive/MyDrive/colab/distillog


In [33]:
%cd drive/MyDrive/colab/distilllog/spirit-kd-chronological

[Errno 2] No such file or directory: 'drive/MyDrive/colab/distilllog/spirit-kd-chronological'
/content


In [10]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.nn.modules.module import Module
from tqdm import tqdm
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import math
import csv
from time import time 
from torchinfo import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [51]:
num_classes = 2
batch_size = 50
learning_rate = 0.1
input_size = 30
sequence_length = 50
hidden_size = 128
num_layers = 2
split = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_path = './datasets/Spirit/chronological_train.csv'
test_path = './datasets/Spirit/chronological_test.csv'
save_path = './datasets/Spirit/model/no-attention-teacher.pth'

In [52]:
class TeacherModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
      super(TeacherModel, self).__init__()
      self.num_layers = num_layers
      self.hidden_size = hidden_size
      self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout = 0.1, batch_first=True)
      self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
      out, _ = self.gru(x)
      out = out[:, -1, :]
      out = self.fc(out)
      return out

class StudentModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
      super(StudentModel, self).__init__()
      self.num_layers = num_layers
      self.hidden_size = hidden_size
      self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout = 0.1, batch_first=True)
      self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
      out, _ = self.gru(x)
      out = out[:, -1, :]
      out = self.fc(out)
      return out

def load_model(model, save_path):
    model.load_state_dict(torch.load(save_path))
    return model

def save_model(model, save_path):
    torch.save(model.state_dict(), save_path)

def mod(l, n):
    """ Truncate or pad a list """
    r = l[-1*n:]
    if len(r) < n:
        r.extend(list([0]) * (n - len(r)))
    return r

def read_data(path, input_size, sequence_length):
    fi = pd.read_csv('./datasets/Spirit/pca_vector_newgenwithfasttext.csv', header = None)
    vec = []
    vec = fi
    vec = np.array(vec)
    
    logs_series = pd.read_csv(path)
    logs_series = logs_series.values
    label = logs_series[:,1]
    logs_data = logs_series[:,0]
    logs = []
    for i in range(0,len(logs_data)):
      ori_seq = [
          int(eventid) for eventid in logs_data[i].split()]
      seq_pattern = mod(ori_seq, sequence_length)
      vec_pattern = []

      for event in seq_pattern:
        if event == 0:
          vec_pattern.append([-1]*input_size)
        else:
          vec_pattern.append(vec[event])  
      logs.append(vec_pattern)
    logs = np.array(logs)
    train_x = logs
    train_y = np.array(label)
    train_x = np.reshape(train_x, (train_x.shape[0], -1, input_size))
    train_y = train_y.astype(int)

    return train_x, train_y

def load_data(train_x, train_y, batch_size):
    tensor_x = torch.Tensor(train_x) 
    tensor_y = torch.from_numpy(train_y)
    train_dataset = TensorDataset(tensor_x,tensor_y) 
    train_loader = DataLoader(train_dataset, batch_size = batch_size) 
    return train_loader


In [53]:

def Teachertrain(model, train_loader, learning_rate, num_epochs):
    criterion = nn.CrossEntropyLoss()
    #summary(model, input_size=(50, 50, 300))
    min_loss = 100
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.0001)  
    model.train()
    
    for epoch in range(num_epochs):
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        total_loss = 0
        for batch_idx, (data, target) in pbar:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item() 
            loss.backward()          
            optimizer.step()
            if total_loss < min_loss:
                min_loss = total_loss
                save_model(Teacher,'./datasets/Spirit/model/no-attention-teacher.pth')

            if (batch_idx+1) % 10 == 0:
                done = (batch_idx+1) * len(data)
                percentage = 100. * batch_idx / len(train_loader)
                pbar.set_description(f'Train Epoch: {epoch+1}/{num_epochs} [{done:5}/{len(train_loader.dataset)} ({percentage:3.0f}%)]  Loss: {total_loss:.6f}')
    return model

In [54]:
fi = pd.read_csv('./datasets/Spirit/pca_vector_newgenwithfasttext.csv', header = None)
vec = []
vec = fi
vec = np.array(vec)

test_logs_series = pd.read_csv(test_path)
test_logs_series = test_logs_series.values
test_total = len(test_logs_series)
sub = int(test_total/split)


def load_test(i):
    if i!=split-1:
        label = test_logs_series[i*sub:(i+1)*sub,1]
        logs_data = test_logs_series[i*sub:(i+1)*sub,0]
    else:
        label = test_logs_series[i*sub:,1]
        logs_data = test_logs_series[i*sub:,0]
    logs = []

    for logid in range(0,len(logs_data)):
        ori_seq = [
            int(eventid) for eventid in logs_data[logid].split()]
        seq_pattern = mod(ori_seq, sequence_length)
        vec_pattern = []

        for event in seq_pattern:
            if event == 0:
                vec_pattern.append([-1]*input_size)
            else:
                vec_pattern.append(vec[event])  
        logs.append(vec_pattern)
    logs = np.array(logs)
    train_x = logs
    train_y = label
    train_x = np.reshape(train_x, (train_x.shape[0], -1, input_size))
    train_y = train_y.astype(int)
    return train_x, train_y



def test(model, criterion = nn.CrossEntropyLoss()):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        TP = 0 
        FP = 0
        FN = 0 
        TN = 0
        for i in range (0, split):        #################################################
            #print(f'batch {i+1}/{split}')
            test_x, test_y = load_test(i)
            test_loader = load_data(test_x, test_y, batch_size)            
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target) # sum up batch loss
                
                output = torch.sigmoid(output)[:, 0].cpu().detach().numpy()
                predicted = (output < 0.2).astype(int)
                target = np.array([y.cpu() for y in target])
                #print(predicted, label)
                TP += ((predicted == 1) * (target == 1)).sum()
                FP += ((predicted == 1) * (target == 0)).sum()
                FN += ((predicted == 0) * (target == 1)).sum()
                TN += ((predicted == 0) * (target == 0)).sum()
        P = 100 * TP / (TP + FP)
        R = 100 * TP / (TP + FN)
        F1 = 2 * P * R / (P + R)   
        accuracy = 100 * (TP + TN)/(TP + TN + FP + FN)  
        #MCC =       
    return accuracy, test_loss, P, R, F1, TP, FP, TN, FN

In [55]:
Teacher = TeacherModel(input_size, hidden_size, num_layers, num_classes).to(device)
summary(Teacher, input_size=(batch_size, sequence_length, input_size))

Layer (type:depth-idx)                   Output Shape              Param #
TeacherModel                             [50, 2]                   --
├─GRU: 1-1                               [50, 50, 128]             160,512
├─Linear: 1-2                            [50, 2]                   258
Total params: 160,770
Trainable params: 160,770
Non-trainable params: 0
Total mult-adds (M): 401.29
Input size (MB): 0.30
Forward/backward pass size (MB): 2.56
Params size (MB): 0.64
Estimated Total Size (MB): 3.50

In [56]:
Student = StudentModel(input_size = 30, hidden_size = 6, num_layers = 2, num_classes = 2).to(device)
#summary(Student, input_size=(batch_size, sequence_length, input_size))

In [57]:
train_x, train_y = read_data(train_path, input_size, sequence_length)
train_loader = load_data(train_x, train_y, batch_size)

In [59]:

#Teacher = load_model(Teacher, save_path)
Teacher = Teachertrain(Teacher, train_loader, learning_rate = 0.0003, num_epochs = 200)


Train Epoch: 1/200 [78400/79999 (100%)]  Loss: 375.610310: 100%|██████████| 1600/1600 [00:06<00:00, 229.73it/s]
Train Epoch: 2/200 [78400/79999 (100%)]  Loss: 164.162434: 100%|██████████| 1600/1600 [00:06<00:00, 252.29it/s]
Train Epoch: 3/200 [78400/79999 (100%)]  Loss: 73.779552: 100%|██████████| 1600/1600 [00:07<00:00, 227.81it/s]
Train Epoch: 4/200 [78400/79999 (100%)]  Loss: 72.683173: 100%|██████████| 1600/1600 [00:06<00:00, 250.50it/s]
Train Epoch: 5/200 [78400/79999 (100%)]  Loss: 65.356550: 100%|██████████| 1600/1600 [00:06<00:00, 250.78it/s]
Train Epoch: 6/200 [78400/79999 (100%)]  Loss: 61.677373: 100%|██████████| 1600/1600 [00:06<00:00, 249.22it/s]
Train Epoch: 7/200 [78400/79999 (100%)]  Loss: 59.327859: 100%|██████████| 1600/1600 [00:06<00:00, 251.97it/s]
Train Epoch: 8/200 [78400/79999 (100%)]  Loss: 56.757760: 100%|██████████| 1600/1600 [00:06<00:00, 248.19it/s]
Train Epoch: 9/200 [78400/79999 (100%)]  Loss: 55.047298: 100%|██████████| 1600/1600 [00:06<00:00, 250.09it/s]

In [60]:
Teacher = load_model(Teacher, save_path)
start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(Teacher, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'
            .format(P, R, F1))

false positive (FP): 141, false negative (FN): 372, true positive (TP): 134, true negative (TN): 19352
Test set: Average loss: 0.0029, Accuracy: 97.43%). Total time = 1.7298221588134766
Precision: 48.727%, Recall: 26.482%, F1-measure: 34.315%


In [ ]:
def train_step(
    Teacher,
    Student,
    optimizer,
    student_loss_fn,
    divergence_loss_fn,
    temp,
    alpha,
    epoch,
    device
):
    losses = []
    pbar = tqdm(train_loader, total=len(train_loader), position=0, leave=True, desc=f"Epoch {epoch+1}")
    for data, targets in pbar:
        # Get data to cuda if possible
        data = data.to(device)
        targets = targets.to(device)

        # forward
        with torch.no_grad():
            teacher_preds = Teacher(data)

        student_preds = Student(data)
        student_loss = student_loss_fn(student_preds, targets)
        
        distillation_loss = divergence_loss_fn(
            F.softmax(student_preds / temp, dim=1),
            F.softmax(teacher_preds / temp, dim=1)
        )
        loss = alpha * student_loss + (1 - alpha) * distillation_loss
        #print(student_loss)
        #print(distillation_loss)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    
    avg_loss = sum(losses) / len(losses)
    return avg_loss

def teach(epochs, Teacher, Student, temp=7, alpha=0.3):
  Teacher = Teacher.to(device)
  Student = Student.to(device)
  student_loss_fn = nn.CrossEntropyLoss()
  divergence_loss_fn = nn.KLDivLoss(reduction="batchmean")
  optimizer = torch.optim.Adam(Student.parameters(), lr=0.01)

  Teacher.eval()
  Student.train()
  for epoch in range(epochs):
      loss = train_step(
          Teacher,
          Student,
          optimizer,
          student_loss_fn,
          divergence_loss_fn,
          temp,
          alpha,
          epoch,
          device
      )

      print(f"Loss:{loss:.2f}")
      

teach(epochs=100, Teacher=Teacher, Student=Student, temp=7, alpha=0.3)


Epoch 1: 100%|██████████| 1600/1600 [00:06<00:00, 257.43it/s]


Loss:-0.82


Epoch 2: 100%|██████████| 1600/1600 [00:06<00:00, 266.38it/s]


Loss:-0.87


Epoch 3: 100%|██████████| 1600/1600 [00:06<00:00, 266.29it/s]


Loss:-0.89


Epoch 4: 100%|██████████| 1600/1600 [00:06<00:00, 262.77it/s]


Loss:-0.90


Epoch 5:  56%|█████▋    | 903/1600 [00:03<00:02, 263.44it/s]

In [ ]:
start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(Student, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'
            .format(P, R, F1))

In [ ]:
noKD = StudentModel(input_size = 30, hidden_size = 6, num_layers = 2, num_classes = 2).to(device) 
noKD = Teachertrain(noKD, train_loader, learning_rate, num_epochs = 100)

In [ ]:
start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(noKD, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'
            .format(P, R, F1))

In [ ]:
save_model(Teacher,'./datasets/Spirit/model/no-attention-teacher.pth')
save_model(Student,'./datasets/Spirit/model/no-attention-student.pth')
save_model(noKD,'./datasets/Spirit/model/no-attention-noKD.pth')